In [1]:
from flow.scenarios import MergeScenario
from flow.controllers import ContinuousRouter, IDMController
from flow.core.params import SumoCarFollowingParams, SumoLaneChangeParams

from flow.core.params import VehicleParams
from flow.controllers import IDMController
from flow.core.params import SumoCarFollowingParams

In [2]:
# create an empty vehicles object
vehicles = VehicleParams()

# add some vehicles to this object of type "human"
vehicles.add(
    veh_id = "human",
    acceleration_controller=(IDMController, {}),
    routing_controller = (ContinuousRouter, {}),
    car_following_params=SumoCarFollowingParams(
        speed_mode="obey_safe_speed",
    ),
    lane_change_params=SumoLaneChangeParams(
        lane_change_mode= "strategic",
    ), num_vehicles = 0)


In [3]:
from flow.core.params import InFlows

inflow = InFlows()

inflow.add(veh_type="human",
           edge="inflow_highway_2",
           vehs_per_hour=2000,
           departSpeed=10,
           departLane="random")

inflow.add(veh_type="human",
           edge="inflow_merge_2",
           vehs_per_hour=500,
           departSpeed=10,
           departLane="random")


In [4]:
from flow.core.params import NetParams

additional_net_params = {
    # length of the merge edge
    "merge_length": 100,
    # length of the highway leading to the merge
    "pre_merge_length": 200,
    # length of the highway past the merge
    "post_merge_length": 100,
    # number of lanes in the merge
    "merge_lanes": 2,
    # number of lanes in the highway
    "highway_lanes": 5,
    # max speed limit of the network
    "speed_limit": 30,
}

# we choose to make the main highway slightly longer
additional_net_params["pre_merge_length"] = 150

net_params = NetParams(inflows=inflow,  # our inflows
                       no_internal_links=False,
                       additional_params=additional_net_params)

In [5]:
from numpy import pi, sin, cos

INFLOW_EDGE_LEN = 200  # length of the inflow edges (needed for resets)
VEHICLE_LENGTH = 5

class customMergeScenario(MergeScenario):
    
    def specify_nodes(self, net_params):
        angle = (2*pi)/3
        smaller_angle = pi / 3
        merge = net_params.additional_params["merge_length"]
        premerge = net_params.additional_params["pre_merge_length"]
        postmerge = net_params.additional_params["post_merge_length"]
        
        nodes = [
            {
                "id": "inflow_highway",
                "x": -INFLOW_EDGE_LEN,
                "y": 0
            },
            {
                "id": "left",
                "y": 0,
                "x": 0
            },
            {
                "id": "center",
                "y": 0,
                "x": premerge,
                "radius": 10
            },
            {
                "id": "right",
                "y": 0,
                "x": premerge + postmerge
            },
            {
                "id": "inflow_merge",
                "x": premerge - (merge + INFLOW_EDGE_LEN) * cos(angle),
                "y": -(merge + INFLOW_EDGE_LEN) * sin(angle)
            },
            {
                "id": "bottom",
                "x": premerge - merge * cos(angle),
                "y": -merge * sin(angle)
            },{
                "id": "center_2",
                "y": 0,
                "x": -INFLOW_EDGE_LEN-1,
                "radius": 10
            },{
                "id": "inflow_merge_2",
                "x": -INFLOW_EDGE_LEN-1 - (merge + INFLOW_EDGE_LEN) * cos(smaller_angle),
                "y": -(merge + INFLOW_EDGE_LEN) * sin(smaller_angle)
            },
            {
                "id": "bottom_2",
                "x": -INFLOW_EDGE_LEN-1 - merge * cos(smaller_angle),
                "y": -merge * sin(smaller_angle)
            },{
                "id": "inflow_highway_2",
                "x": -3*INFLOW_EDGE_LEN,
                "y": 0
            },
            {
                "id": "left_2",
                "y": 0,
                "x": -INFLOW_EDGE_LEN-100
            }
        ]

        return nodes
    
    def specify_edges(self, net_params):
        """See parent class."""
        merge = net_params.additional_params["merge_length"]
        premerge = net_params.additional_params["pre_merge_length"]
        postmerge = net_params.additional_params["post_merge_length"]

        edges = [{
            "id": "inflow_highway",
            "type": "highwayType",
            "from": "inflow_highway",
            "to": "left",
            "length": premerge
        }, {
            "id": "left",
            "type": "highwayType",
            "from": "left",
            "to": "center",
            "length": premerge
        }, {
            "id": "inflow_merge",
            "type": "mergeType",
            "from": "bottom",
            "to": "inflow_merge",
            "length": INFLOW_EDGE_LEN
        }, {
            "id": "bottom",
            "type": "mergeType",
            "from": "center",
            "to": "bottom",
            "length": merge
        }, {
            "id": "center",
            "type": "highwayType",
            "from": "center",
            "to": "right",
            "length": postmerge
        }, 
        {
            "id": "bottom_2",
            "type": "mergeType",
            "from": "bottom_2",
            "to": "inflow_highway",
            "length": merge
        },{
            "id": "inflow_merge_2",
            "type": "mergeType",
            "from": "inflow_merge_2",
            "to": "bottom_2",
            "length": INFLOW_EDGE_LEN
        }, {
            "id": "inflow_highway_2",
            "type": "highwayType",
            "from": "inflow_highway_2",
            "to": "left_2",
            "length": INFLOW_EDGE_LEN
        }, {
            "id": "left_2",
            "type": "highwayType",
            "from": "left_2",
            "to": "inflow_highway",
            "length": premerge
        }]

        return edges 
    
    def specify_edge_starts(self):
        """See parent class."""
        premerge = self.net_params.additional_params["pre_merge_length"]
        postmerge = self.net_params.additional_params["post_merge_length"]

        edgestarts = [("inflow_highway", 0), ("left", INFLOW_EDGE_LEN + 0.1),
                      ("center", INFLOW_EDGE_LEN + premerge + 22.6),
                      ("inflow_merge",
                       INFLOW_EDGE_LEN + premerge + postmerge + 22.6),
                      ("bottom",
                       2 * INFLOW_EDGE_LEN + premerge + postmerge + 22.7),
                    #  ("center_2", -1),
                     ("inflow_merge_2", -1),
                     ("bottom_2", -2),
                     ("inflow_highway_2", -3),
                     ("left_2", -INFLOW_EDGE_LEN + 0.1)]

        return edgestarts

    def specify_internal_edge_starts(self):
        """See parent class."""
        premerge = self.net_params.additional_params["pre_merge_length"]
        postmerge = self.net_params.additional_params["post_merge_length"]

        internal_edgestarts = [
            (":left", INFLOW_EDGE_LEN), (":center",
                                         INFLOW_EDGE_LEN + premerge + 0.1),
            (":bottom", 2 * INFLOW_EDGE_LEN + premerge + postmerge + 22.6)
        ]

        return internal_edgestarts
    
    def specify_routes(self, net_params):
        """See parent class."""
        rts = {
            "inflow_highway": ["inflow_highway", "left"],
            "left": [(["left", "bottom"], .3), (["left", "center"], .7)],
            "center": ["center"],
            "inflow_merge": ["inflow_merge"],
            "bottom": ["bottom", "inflow_merge"],
            "inflow_merge_2": ["inflow_merge_2", "bottom_2"],
            "bottom_2": ["bottom_2", "inflow_highway"],
            "inflow_highway_2": ["inflow_highway_2", "left_2"],
            "left_2": ["left_2", "inflow_highway"]
        }

        return rts


In [6]:
from flow.core.params import SumoParams, EnvParams, InitialConfig
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.core.experiment import Experiment

sumo_params = SumoParams(render=True,
                         sim_step=0.2)

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

initial_config = InitialConfig(spacing="random", perturbation=1)

scenario = customMergeScenario(
    name="custom-triangle-merge-example",
    vehicles=vehicles,
    net_params=net_params,
    initial_config=initial_config
)

# test_scenario = MergeScenario(name="merge-example",
#                          vehicles=vehicles,
#                          net_params=net_params,
#                          initial_config=initial_config)

env = AccelEnv(env_params, sumo_params, scenario)

exp = Experiment(env)

_ = exp.run(1, 10000)


**********************************************************
**********************************************************
**********************************************************
significantly decrease after large number of rollouts. In 
order to avoid this, set SumoParams(restart_instance=True).
**********************************************************
**********************************************************
**********************************************************


/home/flow/anaconda3/envs/flow-master/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/flow/anaconda3/envs/flow-master/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Round 0, return: 412.3503692072843
Average, std return: 412.3503692072843, 0.0
Average, std speed: nan, nan
